In [1]:
%matplotlib inline
import sys
import pandas as pd
import math


from __future__ import print_function
from scipy.stats import entropy 

# turn of data table rendering
pd.set_option('display.notebook_repr_html', False)
sys.version

'3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]'

## Example
We want to use a toy data set with some features to predict whether a person is happy or not. We want to build an efficient decision tree to predict the label based on new data. In this example, we examine Shannon's Entropy and Information Gain concepts to efficiently separate data points based on feature nodes.

In [2]:
df = pd.read_csv('data/entropy.csv')
df

  gender personality work  happiness
0    man        rude   no  not happy
1    man        nice   no  not happy
2  woman        rude   no      happy
3    man        nice  yes      happy
4  woman        nice  yes      happy

## Entropy
Entropy is a measure of unpredictability of information content expressed in bits. If we build a [decision tree](http://unsupervised-learning.com/decision-tree-classifier-on-guitar-models-in-python/), for instance, we first have to choose the starting node on which to split the data. We do this by determining the information gain of each feature we have. The more information we gain by splitting the data on a certain feature the better. [Claude Shannon's Entropy Formula](http://crackingthenutshell.com/what-is-information-part-2a-information-theory/) where entropy is denoted as $H$:

$$
H(X)=-\sum_{i=1}^n p(X_i)\cdot log_2p(X_i)
$$

In this example, it is the negative sum over all the data points of a label $X$ (outcome) times the log base 2 of all data points of that label. Here's an example of calculating the entropy of the happiness labels from our data set, where each possible label has a probability of $0.5$:

$$
H(X)=-p(x_0)\cdot log_2p(x_0)-p(x_1)\cdot log_2p(x_1)
$$

$$
H(happiness)=-p(happy)\cdot log_2p(happy)-p(not\ happy)\cdot log_2p(not\ happy)\\
H(happiness)=-0.5\cdot log_2(0.5)-0.5\cdot log_2(0.5)=1.0\\
$$

In code it looks like this:

In [3]:
# Calculate the happiness label entropy
n = len(df)  # number of data points
n_happy = float(df.happiness.value_counts().happy)
p_happy = n_happy / n  # probability of the happy label in the data set
p_nothappy = 1.0 - p_happy  # complementary label

entropy_happiness = -p_happy * math.log(p_happy, 2) - p_nothappy * math.log(p_nothappy, 2)
entropy_happiness

0.9709505944546686

This set of labels is impure since it contains a variety of the labels (happy and not happy). But what if we had four happy and only one not happy label?

In [4]:
# Calculate the happiness label entropy
p_happy = 4.0 / n  # higher probability of the happy label
p_nothappy = 1.0 - p_happy  # lower value for the complementary label

entropy_happiness_2 = -p_happy * math.log(p_happy, 2) - p_nothappy * math.log(p_nothappy, 2)
entropy_happiness_2

0.7219280948873623

The label happiness has now a lower entropy. The data is stil not completely pure, but better than before. So what if we had only happy labels?

In [5]:
# Calculate the happiness label entropy
p_happy = 5.0 / n  # all labels are happy now
p_nothappy = 1.0 - p_happy  # complementary label is zero now

entropy_happiness_3 = -p_happy * math.log(p_happy, 2) - 0
entropy_happiness_3

-0.0

We get an entropy of zero, because the data points for happiness all have the same value; happy. This is the purest state of the data. In other words, it has the least amount of uncertainty.

In [6]:
# Another, more convenient, way to calculate entropy is by using 
# the entropy function from the scipy.stats library
p_happy = .75
p_nothappy = .25

entropy([p_happy, p_nothappy], base=2)

0.8112781244591328

## Information Gain
We use Shannon's concept of information Gain to decide which feature to use to split the data in our decision tree. The goal is the choose the feature that maximizes the information gain. The information gain is calculated with the following formula:

$$
information\ gain=entropy(parent)- \begin{bmatrix}weighted\\average\end{bmatrix}entropy(children)
$$

In [7]:
# First calculate the entropy of happiness if we split
# the data based on gender=man
n_man = 3.0  # three labels left after splitting
p_man_happy = 1.0 / n_man  # one happy label
p_man_nothappy = 2.0 / n_man  # two not happy labels

entropy_man_happiness = entropy([p_man_happy, p_man_nothappy], base=2)
entropy_man_happiness

0.9182958340544894

In [8]:
# Then calculate the entropy of happiness if we split
# the data based on gender=woman
n_woman = 2.0  # two labels left after splitting
p_woman_happy = 1.0 / n_woman  # one happy label
p_woman_nothappy = 1.0 / n_woman # one not happy label

entropy_woman_happiness = entropy([p_woman_happy, p_woman_nothappy], base=2)
entropy_woman_happiness

1.0

In [9]:
# Calculate the weighted average of the entropy of the children
happiness_split_by_man = 3.0 / 5.0
happiness_split_by_woman = 1 - happiness_split_by_man

weighted_avg_entropy_children = \
    (happiness_split_by_man * entropy_man_happiness - \
     happiness_split_by_woman * entropy_woman_happiness)
weighted_avg_entropy_children

0.15097750043269365

In [10]:
# Calculate the information gain if we split based on gender
entropy_happiness = 0.9709505944546686  # hard coded entropy of the parent
information_gain_split_gender = entropy_happiness - weighted_avg_entropy_children
information_gain_split_gender

0.8199730940219749

If we build a decision tree, the information gain value is relative to the other possible ways to split the data. Therefore, we need to calculate the other features as well. A quick calculation gives us the information gain values if we split the complete data set based on the following features:

|Feature|Information Gain|
|:---|:---|
|gender|0.81997309402197494|
|personality|0.9709505944546686|
|work|0.9709505944546686|

With this result, we can either split on personality or work since they have both the largest information gain.

### All Done.

#### Next: Go out and play! :)